In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from keras.models import load_model, save_model, Sequential
from keras.utils import get_file, to_categorical
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping
from imblearn.over_sampling import SMOTE
from collections import Counter
import requests
import os
import io
from scipy.stats import zscore

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_columns', 100)

In [3]:
df = pd.read_csv('CSV\KDDCUP99\kdd-with-columns.csv')
# df = pd.read_csv('CSV\KDDCUP99\kddcup-data-original.csv')

# sample only 10% of the dataset
# df = df.sample(frac=0.1, replace=False) # Uncomment this line to 


In [4]:
has_null_or_nan = df.isnull().values.any()

is_empty = df.empty

result = has_null_or_nan or is_empty

print(result)

# df.dropna(inplace=True,axis=1)

False


In [5]:
# df.columns = df_temp.columns

df['outcome'] = df['outcome'].apply(lambda x: x.rstrip('.'))

# del df_temp

In [6]:
df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,outcome
0,0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.0,0.0,1.0,0.0,0.00,9,9,1.0,0.0,0.11,0.00,0.00,0.00,0.0,0.0,normal
1,0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.0,0.0,1.0,0.0,0.00,19,19,1.0,0.0,0.05,0.00,0.00,0.00,0.0,0.0,normal
2,0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.0,0.0,1.0,0.0,0.00,29,29,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal
3,0,tcp,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.0,0.0,1.0,0.0,0.00,39,39,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal
4,0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.0,0.0,1.0,0.0,0.00,49,49,1.0,0.0,0.02,0.00,0.00,0.00,0.0,0.0,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,0,tcp,http,SF,310,1881,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,5,0.00,0.00,0.0,0.0,1.0,0.0,0.40,86,255,1.0,0.0,0.01,0.05,0.00,0.01,0.0,0.0,normal
494017,0,tcp,http,SF,282,2286,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.0,0.0,1.0,0.0,0.00,6,255,1.0,0.0,0.17,0.05,0.00,0.01,0.0,0.0,normal
494018,0,tcp,http,SF,203,1200,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,18,0.17,0.11,0.0,0.0,1.0,0.0,0.17,16,255,1.0,0.0,0.06,0.05,0.06,0.01,0.0,0.0,normal
494019,0,tcp,http,SF,291,1200,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,12,0.00,0.00,0.0,0.0,1.0,0.0,0.17,26,255,1.0,0.0,0.04,0.05,0.04,0.01,0.0,0.0,normal


## Analyzing a Dataset

In [7]:
def expand_categories(values):
    result = []
    s = values.value_counts()
    t = float(len(values))
    for v in s.index:
        result.append("{}:{}%".format(v,round(100*(s[v]/t),2)))
    return "[{}]".format(",".join(result))
        
def analyze(df):
    print()
    cols = df.columns.values
    total = float(len(df))

    print("{} rows".format(int(total)))
    for col in cols:
        uniques = df[col].unique()
        unique_count = len(uniques)
        if unique_count>50:
            print("** {}:{} ({}%)".format(col,unique_count, int(((unique_count)/total)*100)))
        else:
            print("** {}:{}".format(col,expand_categories(df[col])))
            expand_categories(df[col])

In [8]:
print(df.groupby('outcome')['outcome'].count())
# print(df_temp.groupby('outcome')['outcome'].count())

outcome
back                 2203
buffer_overflow        30
ftp_write               8
guess_passwd           53
imap                   12
ipsweep              1247
land                   21
loadmodule              9
multihop                7
neptune            107201
nmap                  231
normal              97278
perl                    3
phf                     4
pod                   264
portsweep            1040
rootkit                10
satan                1589
smurf              280790
spy                     2
teardrop              979
warezclient          1020
warezmaster            20
Name: outcome, dtype: int64


In [9]:
analyze(df)


494021 rows
** duration:2495 (0%)
** protocol_type:[icmp:57.41%,tcp:38.47%,udp:4.12%]
** service:66 (0%)
** flag:[SF:76.6%,S0:17.61%,REJ:5.44%,RSTR:0.18%,RSTO:0.12%,SH:0.02%,S1:0.01%,S2:0.0%,RSTOS0:0.0%,S3:0.0%,OTH:0.0%]
** src_bytes:3300 (0%)
** dst_bytes:10725 (2%)
** land:[0:100.0%,1:0.0%]
** wrong_fragment:[0:99.75%,3:0.2%,1:0.05%]
** urgent:[0:100.0%,1:0.0%,2:0.0%,3:0.0%]
** hot:[0:99.35%,2:0.44%,28:0.06%,1:0.05%,4:0.02%,6:0.02%,5:0.01%,3:0.01%,14:0.01%,30:0.01%,22:0.01%,19:0.0%,24:0.0%,18:0.0%,20:0.0%,7:0.0%,17:0.0%,12:0.0%,16:0.0%,10:0.0%,15:0.0%,9:0.0%]
** num_failed_logins:[0:99.99%,1:0.01%,2:0.0%,5:0.0%,4:0.0%,3:0.0%]
** logged_in:[0:85.18%,1:14.82%]
** num_compromised:[0:99.55%,1:0.44%,2:0.0%,4:0.0%,3:0.0%,6:0.0%,5:0.0%,7:0.0%,767:0.0%,12:0.0%,9:0.0%,884:0.0%,13:0.0%,38:0.0%,18:0.0%,11:0.0%,275:0.0%,281:0.0%,16:0.0%,238:0.0%,21:0.0%,22:0.0%,102:0.0%]
** root_shell:[0:99.99%,1:0.01%]
** su_attempted:[0:100.0%,1:0.0%,2:0.0%]
** num_root:[0:99.88%,1:0.05%,9:0.03%,6:0.03%,2:0.0

## Encode the feature vector

In [10]:
# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    if sd == 0:
        df[name] = df[name] - mean
    else:
        df[name] = (df[name] - mean) / sd
    
# Encode text values to dummy variables(i.e. [1,0,0],
# [0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    df[name].fillna('Unknown', inplace=True)
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


In [11]:
# Now encode the feature vector

for name in df.columns:
  if name == 'outcome':
    pass
  elif name in ['protocol_type','service','flag','land','logged_in','is_host_login','is_guest_login']:
    encode_text_dummy(df,name)
  else:
    encode_numeric_zscore(df,name)    

# Convert to numpy - Classification
x_columns = df.columns.drop('outcome')
x = df[x_columns].values
dummies = pd.get_dummies(df['outcome']) # Classification
outcomes = dummies.columns
num_classes = len(outcomes)
y = dummies.values

y_df = pd.DataFrame(y, columns=outcomes)
y_df

C:\Users\hhi20\AppData\Local\Temp\ipykernel_27412\1867040172.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[name].fillna('Unknown', inplace=True)
C:\Users\hhi20\AppData\Local\Temp\ipykernel_27412\1867040172.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

,back,buffer_overflow,ftp_write,guess_passwd,imap,ipsweep,land,loadmodule,multihop,neptune,nmap,normal,perl,phf,pod,portsweep,rootkit,satan,smurf,spy,teardrop,warezclient,warezmaster
0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
494017,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
494018,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
494019,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False


In [12]:
has_null_or_nan = pd.DataFrame(x).isnull().values.any()

is_empty = pd.DataFrame(x).empty

result = has_null_or_nan or is_empty

print(result)

# df.dropna(inplace=True,axis=1)

False


In [13]:
# y_df = pd.DataFrame(y, columns=outcomes)
# y_df

In [14]:
# df.loc[df['protocol_type']=='tcp', 'protocol_type'] =1
# df.loc[df['protocol_type']=='udp', 'protocol_type'] =2
# df.loc[df['protocol_type']=='icmp', 'protocol_type']=3

In [15]:
# df.loc[df['flag']== 'SF', 'flag'] = 1
# df.loc[df['flag']== 'S0', 'flag'] = 2
# df.loc[df['flag']== 'REJ', 'flag'] = 3
# df.loc[df['flag']== 'RSTR', 'flag'] = 4
# df.loc[df['flag']== 'SH', 'flag'] = 5
# df.loc[df['flag']== 'RSTO', 'flag'] = 6
# df.loc[df['flag']== 'S1', 'flag'] = 7
# df.loc[df['flag']== 'RSTOS0', 'flag'] = 8
# df.loc[df['flag']== 'S3', 'flag'] = 9
# df.loc[df['flag']== 'S2', 'flag'] = 10
# df.loc[df['flag']== 'OTH', 'flag'] = 11

In [16]:
# df.loc[df['outcome']=='buffer_overflow.', 'outcome'] = 'other attack'
# df.loc[df['outcome']=='ftp_write.', 'outcome'] = 'other attack'
# df.loc[df['outcome']=='imap.', 'outcome'] = 'other attack'
# df.loc[df['outcome']=='land.', 'outcome'] = 'other attack'
# df.loc[df['outcome']=='loadmodule.', 'outcome'] = 'other attack'
# df.loc[df['outcome']=='multihop.', 'outcome'] = 'other attack'
# df.loc[df['outcome']=='perl.', 'outcome'] = 'other attack'
# df.loc[df['outcome']=='phf.', 'outcome'] = 'other attack'
# df.loc[df['outcome']=='rootkit.', 'outcome'] = 'other attack'
# df.loc[df['outcome']=='spy.', 'outcome'] = 'other attack'
# df.loc[df['outcome']=='warezmaster.', 'outcome'] = 'other attack'


In [17]:
# df.groupby('outcome')['outcome'].count()

In [18]:
# x = df[['protocol_type','flag','src_bytes','dst_bytes','hot','count','srv_count','same_srv_rate','dst_host_count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_rerror_rate']]
# x

In [21]:
oversample = SMOTE(k_neighbors=1)
x,y = oversample.fit_resample(x,y)

# Convert one-hot encoded labels back to class labels
y_class_labels = np.argmax(y, axis=1)

# Count the number of instances for each class
class_counts = Counter(y_class_labels)

print('Classes and number of values in trainset after SMOTE:',class_counts)

Classes and number of values in trainset after SMOTE: Counter({11: 280790, 1: 280790, 7: 280790, 12: 280790, 9: 280790, 18: 280790, 3: 280790, 14: 280790, 20: 280790, 15: 280790, 5: 280790, 6: 280790, 2: 280790, 0: 280790, 4: 280790, 17: 280790, 13: 280790, 10: 280790, 8: 280790, 22: 280790, 21: 280790, 19: 280790, 16: 280790})


In [22]:
outcomes

Index(['back', 'buffer_overflow', 'ftp_write', 'guess_passwd', 'imap',
       'ipsweep', 'land', 'loadmodule', 'multihop', 'neptune', 'nmap',
       'normal', 'perl', 'phf', 'pod', 'portsweep', 'rootkit', 'satan',
       'smurf', 'spy', 'teardrop', 'warezclient', 'warezmaster'],
      dtype='object')

## Train the Neural Network

In [23]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

x_train = np.asarray(x_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')
x_test = np.asarray(x_test).astype('float32')
y_test = np.asarray(y_test).astype('float32')

# Create neural net
model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(50, input_dim=x.shape[1], activation='relu'))
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto', restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test), callbacks=[monitor],verbose=2,epochs=25)
        

Epoch 1/25
161455/161455 - 713s - loss: 0.8316 - val_loss: 0.5729 - 713s/epoch - 4ms/step
Epoch 2/25
161455/161455 - 323s - loss: 0.5213 - val_loss: 0.4686 - 323s/epoch - 2ms/step
Epoch 3/25
161455/161455 - 321s - loss: 0.4128 - val_loss: 0.4326 - 321s/epoch - 2ms/step
Epoch 4/25
161455/161455 - 335s - loss: 0.3829 - val_loss: 0.3727 - 335s/epoch - 2ms/step
Epoch 5/25
161455/161455 - 330s - loss: 0.3329 - val_loss: 0.2565 - 330s/epoch - 2ms/step
Epoch 6/25
161455/161455 - 318s - loss: 0.2557 - val_loss: 0.1582 - 318s/epoch - 2ms/step
Epoch 7/25
161455/161455 - 317s - loss: 0.2311 - val_loss: 0.2392 - 317s/epoch - 2ms/step
Epoch 8/25
161455/161455 - 330s - loss: 0.2370 - val_loss: 0.1393 - 330s/epoch - 2ms/step
Epoch 9/25
161455/161455 - 345s - loss: 0.1981 - val_loss: 0.2681 - 345s/epoch - 2ms/step
Epoch 10/25
161455/161455 - 350s - loss: 0.1876 - val_loss: 0.1580 - 350s/epoch - 2ms/step
Epoch 11/25
161455/161455 - 351s - loss: 0.1796 - val_loss: 0.1305 - 351s/epoch - 2ms/step
Epoch 12

In [24]:
# Measure accuracy
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(y_eval, pred)
print("Validation score: {}".format(score))

40364/40364 [==============================] - 55s 1ms/step
Validation score: 0.9967312721715285


In [25]:
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

y_test = np.argmax(y_test, axis=1)

report = classification_report(y_test, y_pred)
print(report)

40364/40364 [==============================] - 57s 1ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56501
           1       0.98      1.00      0.99     56019
           2       0.99      1.00      1.00     55860
           3       1.00      1.00      1.00     56035
           4       1.00      1.00      1.00     56186
           5       0.99      0.99      0.99     56378
           6       1.00      1.00      1.00     56217
           7       1.00      1.00      1.00     56037
           8       0.99      1.00      0.99     56308
           9       1.00      1.00      1.00     55941
          10       0.99      0.99      0.99     56151
          11       1.00      0.97      0.99     56418
          12       1.00      1.00      1.00     56345
          13       1.00      1.00      1.00     56118
          14       1.00      1.00      1.00     56004
          15       1.00      1.00      1.00     56104
          16       1.

In [26]:
save_model(model, 'kddDnn.h5')

with open('outcomes.pkl', 'wb') as f:
    pickle.dump(outcomes, f)
# # loaded_model = load_model('kddDnn.h5')

In [27]:
# import tensorflow as tf
# print(tf.__version__)

# import keras
# print(keras.__version__)

# import sys
# print(sys.version)